In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Business Problem 
**Objective:** To ranks the stocks from highest to lowest expected returns and is evaluated on the difference in returns between the top and bottom 200 stocks


## Domain Knowledge 

**Evaluation:**  the Sharpe Ratio of the daily spread returns.

In [ ]:
import numpy as np
import pandas as pd
import jpx_tokyo_market_prediction
from lightgbm import LGBMRegressor

import matplotlib.pyplot as plt
import os
import pandas as pd 
import numpy as np

import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor

import warnings
warnings.filterwarnings("ignore")

### Exploring all the datasets 

### Train Files

In [ ]:
financial = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/train_files/financials.csv")
options = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/train_files/options.csv")
ssp = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/train_files/secondary_stock_prices.csv")

trades = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/train_files/trades.csv")
stock = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv")
stock_list=pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/stock_list.csv")

### Stocks :
- This is the primary file 
- The target variables helps us to rank the stocks 
- This is the file which will be used for prediction


In [ ]:
print("Total number of stocks given:" , len(np.unique(stock["SecuritiesCode"])))


In [ ]:
stock.groupby(["Date"])["Open","High","Low","Close"].median().plot(figsize =(15,5))
plt.title("General Trend of stock Open, High, Low, Close")
plt.show()

In [ ]:
stock.groupby(["Date"])["Volume"].median().plot(figsize =(15,5))
plt.title("General Trend of Volumes")
plt.show()

In [ ]:
#stock.groupby(["AdjustmentFactor"]).count()["RowId"]
#stock['Section/Products'] = stock["SecuritiesCode"].map(dict(stock_list[['SecuritiesCode', 'Section/Products']].values))
#stock['NewMarketSegment'] = stock["SecuritiesCode"].map(dict(stock_list[['SecuritiesCode', 'NewMarketSegment']].values))
#stock['IssuedShares'] = stock["SecuritiesCode"].map(dict(stock_list[['SecuritiesCode', 'IssuedShares']].values))
#stock['MarketCapitalization'] = stock["SecuritiesCode"].map(dict(stock_list[['SecuritiesCode', 'MarketCapitalization']].values))

In [ ]:
class Tech_indicators:
    def __init__(self,df, column):
        self.df = df
   
        self.column = column
        
     ### Simple Moving Average    
    def SMA(self):
        
        return self.df[self.column].rolling(window=5).mean(),self.df[self.column].rolling(window=15).mean(),self.df[self.column].rolling(window=50).mean(),self.df[self.column].rolling(window=200).mean()
                
    ### Exponential Moving Avergae 
    def EMA(self):
        return self.df[self.column].ewm(span=5).mean(),self.df[self.column].ewm(span=15).mean(), self.df[self.column].ewm(span=50).mean(), self.df[self.column].ewm(span=200).mean()
               
              
    def volume_50(self):
         return self.df["Volume"].rolling(window=50).mean()
        
               
               
    
    ##OBV
    def OBV(self):

            #self.df.loc[:,"OBV"] = 0
            self.df["OBV"] = np.nan
           
            neg_change = self.df["Volume"] < self.df["Volume"].shift(1)
            pos_change = self.df["Volume"] >= self.df["Volume"].shift(1)
            no_change = self.df["Volume"] == self.df["Volume"].shift(1)

            if pos_change.any():
                self.df.loc[pos_change, "OBV"] = self.df["Volume"]
            if neg_change.any():
                self.df.loc[neg_change, "OBV"] = -self.df["Volume"]
            if no_change.any():
                self.df.loc[no_change, "OBV"] = self.df["Volume"].shift(1)


            return pd.Series(self.df["OBV"].cumsum())
        
###Stochastic Oscillator

    def STOCH(self):
        highest_high = self.df["High"].rolling(center=False, window=14).max()
        lowest_low = self.df["Low"].rolling(center=False, window=14).min()
        STOCH = pd.Series((self.df["Close"] - lowest_low) / (highest_high - lowest_low) * 100)
        return STOCH
    
    
    def RSI(self):
        delta = self.df["Close"].diff()

        ## positive gains (up) and negative gains (down) Series
        up, down = delta.copy(), delta.copy()
        up[up < 0] = 0
        down[down > 0] = 0

        # EMAs of ups and downs
        _gain = up.ewm(alpha=1.0 / 14 ).mean()
        _loss = down.abs().ewm(alpha=1.0 / 14).mean()

        RS = _gain / _loss
        return pd.Series(100 - (100 / (1 + RS)))
        

### I will add more Technical Indicators and explain indicators 
    


In [ ]:
from tqdm import tqdm 

In [ ]:
mega_df = pd.DataFrame()
for i in tqdm(np.unique(stock["SecuritiesCode"])):
    df = stock[stock["SecuritiesCode"] == i]
    M1 = Tech_indicators(df, "Close")
    df["EMA_5"] = M1.EMA()[0]
    df["EMA_15"] = M1.EMA()[1]
    df["EMA_50"] = M1.EMA()[2]
    df["EMA_200"] = M1.EMA()[3]
    df["STOCH"] = M1.STOCH()
    df["OBV"] = M1.OBV()
    mega_df = pd.concat([mega_df,df])
    

In [ ]:
df = mega_df[mega_df["SecuritiesCode"]==1332]
df["Date"] = pd.to_datetime(df["Date"])
df_1year = df[df["Date"] > "2020-12-03"]

In [ ]:
stock_1332_1year = df_1year

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go


fig3 = make_subplots(specs=[[{"secondary_y": True}]])
fig3.add_trace(go.Candlestick(x=stock_1332_1year["Date"],
                              open=stock_1332_1year['Open'],
                              high=stock_1332_1year['High'],
                              low=stock_1332_1year['Low'],
                              close=stock_1332_1year['Close'], name="Candlestick"))


fig3.add_trace(go.Scatter(x=stock_1332_1year["Date"],y=stock_1332_1year['EMA_5'],marker_color='magenta',name='5 Day EMA'))
fig3.add_trace(go.Scatter(x=stock_1332_1year["Date"],y=stock_1332_1year['EMA_15'],marker_color='blue',name='15 Day EMA'))

fig3.add_trace(go.Scatter(x=stock_1332_1year["Date"],y=stock_1332_1year['EMA_50'],marker_color='green',name='50 Day EMA'))
fig3.add_trace(go.Scatter(x=stock_1332_1year["Date"],y=stock_1332_1year['EMA_200'],marker_color='red',name='200 Day EMA'))

fig3.add_trace(go.Bar(x=stock_1332_1year["Date"], y=stock_1332_1year['Volume'], name='Volume'),secondary_y=True)
fig3.update_yaxes(range=[0,35000000],secondary_y=True)
fig3.update_yaxes(visible=False, secondary_y=True)
fig3.update_layout(xaxis_rangeslider_visible=False)  
fig3.show()

- When the line of 5 EMA crosses 15 EMA from below and even 50 EMA and then 200 EMA with high volumes ,the stock prices rises 
- When the 5 EMA crosses 15 EMA from above we can observe the stock price is relatively stable 
- We also observe on OCT 14 the stock price rises with high volume and it is not able to cross the high level which is 689 with same volume or more , which is indicates the stock has lost its strenght 

In [ ]:
mega_df["Date"] = pd.to_datetime(mega_df["Date"])
mega_df["Year"]=mega_df["Date"].dt.year
mega_df["Day"]=mega_df["Date"].dt.day
mega_df["Month"]= mega_df["Date"].dt.month
mega_df = mega_df.fillna(0)
y = mega_df["Target"]
mega_df = mega_df.drop(["RowId","Date","Target"],axis=1)
mega_df["SupervisionFlag"] = mega_df["SupervisionFlag"].map(lambda x : int(x))

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(mega_df, y, test_size=0.2, shuffle=True, random_state=0) # train, valid 8:2 분할

In [ ]:
params = {
          }

model = LGBMRegressor(**params)

model.fit(X_train, 
          y_train, 
          eval_set = [(X_train, y_train)],
          eval_metric = 'mse')



from sklearn.metrics import mean_squared_error
oof_preds = model.predict(X_valid)
oof_score = np.sqrt(mean_squared_error(y_valid, oof_preds))
print(oof_score)